Tatoebaコーパスをダウンロードし、前処理するところを自分の手でやりたい場合は上から２番目のセルを実行する。スキップしたい場合は、上から２番目のセルだけ飛ばして実行する。

ただし、前処理済みのデータセットが既に用意されているので、必ずしも実行する必要はない。

In [ ]:
import os

os.chdir("/")
!git clone https://github.com/hir777/Machine_Translation_Proto.git
os.chdir("/Machine_Translation_Proto/")
!pip install -r requirements.txt

fatal: destination path 'Machine_Translation_Proto' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python3 -m unidic download

download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:28<00:00, 18.4MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.7/dist-packages/unidic/dicdir


ここまで実行できたら、一度ランタイムを再起動する。
その際、上のセルを忘れずに再実行する。

In [ ]:
!nvidia-smi

Sun Jun 26 04:44:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
os.chdir("/Machine_Translation_Proto/corpus/src")

!python3 create_dataset.py \
--repo_path /Machine_Translation_Proto/ \
--workers_clean 8 \
--workers_tkn 12 \
--workers_freq 8 \
--cleaning \
--freq_filter \
--ratio_filter \
--len_filter \
--overlap_filter \
--tatoeba \
--div_train \
--div_valid \
--div_test

/Machine_Translation_Proto/corpus/data
Using custom data configuration en-ja-lang1=en,lang2=ja
Reusing dataset tatoeba (/root/.cache/huggingface/datasets/tatoeba/en-ja-lang1=en,lang2=ja/0.0.0/b3ea9c6bb2af47699c5fc0a155643f5a0da287c7095ea14824ee0a8afd74daf6)
100% 1/1 [00:00<00:00, 538.35it/s]
Creating json from Arrow format: 100% 21/21 [00:00<00:00, 26.25ba/s]

Reformating a json file downloaded from tatoeba...
100% 208866/208866 [00:00<00:00, 802597.76it/s]

Converting a json file into list...
100% 208866/208866 [00:00<00:00, 2112683.61it/s]
Start denoising sentences... (Process ID: 1110)
Start denoising sentences... (Process ID: 1111)
Start denoising sentences... (Process ID: 1112)
Start denoising sentences... (Process ID: 1113)
Start denoising sentences... (Process ID: 1114)
Start denoising sentences... (Process ID: 1115)
Start denoising sentences... (Process ID: 1116)
Start denoising sentences... (Process ID: 1117)
Finished denoising sentences... (Process ID: 1110)
Finished denoisin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/checkpoints"

In [ ]:
os.chdir("/Machine_Translation_Proto/corpus/data/")
!cat train*.en >> train.en
!cat train*.ja >> train.ja
!cat valid*.en >> valid.en
!cat valid*.ja >> valid.ja
!cat test*.en >> test.en
!cat test*.ja >> test.ja

cat: train.en: input file is output file
cat: 'train*.ja': No such file or directory
cat: valid.en: input file is output file
cat: valid.ja: input file is output file
cat: test.en: input file is output file
cat: test.ja: input file is output file


Tatoebaデータセット

In [ ]:
os.chdir("/Machine_Translation_Proto/corpus/data/")
!mv train1.en train.en
!mv train1.ja train.ja
!mv valid1.en valid.en
!mv valid1.ja valid.ja
!mv test1.en test.en
!mv test1.ja test.ja

In [ ]:
!wc -l train.en train.ja valid.en valid.ja test.en test.ja

In [ ]:
os.chdir("/Machine_Translation_Proto/")
!bash pre-process.sh REPO_PATH="/Machine_Translation_Proto"

+ for ARGUMENT in "$@"
++ echo REPO_PATH=/Machine_Translation_Proto
++ cut -f1 -d=
+ KEY=REPO_PATH
+ KEY_LENGTH=9
+ VALUE=/Machine_Translation_Proto
+ export REPO_PATH=/Machine_Translation_Proto
+ REPO_PATH=/Machine_Translation_Proto
+ TRAIN_EN=/Machine_Translation_Proto/corpus/data/train.en
+ TRAIN_JA=/Machine_Translation_Proto/corpus/data/train.ja
+ VALID_EN=/Machine_Translation_Proto/corpus/data/valid.en
+ VALID_JA=/Machine_Translation_Proto/corpus/data/valid.ja
+ TEST_EN=/Machine_Translation_Proto/corpus/data/test.en
+ TEST_JA=/Machine_Translation_Proto/corpus/data/test.ja
+ TRAIN_SP=/Machine_Translation_Proto/src/train_sp.py
+ ENCODE=/Machine_Translation_Proto/src/encode.py
+ cat /Machine_Translation_Proto/corpus/data/train.en /Machine_Translation_Proto/corpus/data/train.ja
+ python /Machine_Translation_Proto/src/train_sp.py --input train.enja --prefix bpe --vocab_size 8000 --character_coverage 0.9995
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  

In [ ]:
!fairseq-train \
    /Machine_Translation_Proto/data-bin \
    --fp16 \
    --save-interval 5 \
    --save-dir "/content/drive/MyDrive/checkpoints/"
    --log-interval 1 \
    --log-format simple \
    --max-epoch 120 \
    --update-freq 16 \
    --max-update 30000 \
    --max-tokens 4000 \
    --arch transformer \
    --share-all-embeddings \
    --encoder-embed-dim 512 \
    --encoder-ffn-embed-dim 4096 \
    --encoder-attention-heads 8 \
    --encoder-layers 6 \
    --decoder-embed-dim 512 \
    --decoder-ffn-embed-dim 4096 \
    --decoder-attention-heads 8 \
    --decoder-layers 6 \
    --share-all-embeddings \
    --dropout 0.3 \
    --activation-fn gelu \
    --optimizer adam \
    --adam-betas '(0.9, 0.98)' \
    --lr 0.001 \
    --lr-scheduler inverse_sqrt \
    --scoring sacrebleu \
    --warmup-updates 4000 \
    --warmup-init-lr 1e-07 \
    --clip-norm 0.0 \
    --weight-decay 0.0 \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --task translation

2022-06-26 04:51:54 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.1, activation_fn='gelu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.1, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=1.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='/Machine_Translation_Proto/data-bin', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=4096, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=8, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=True, decoder_output_dim=512, device_id=0, disable_valida

チェックポイントを指定し、モデルの学習を再開する。
（データセットは同じものを用いる）

In [ ]:
!cp "/Machine_Translation_Proto/checkpoints/checkpoint1.pt" "/content/drive/MyDrive/checkpoints/" 

In [ ]:
!fairseq-train \
    /Machine_Translation_Proto/data-bin \
    --fp16 \
    --restore-file checkpoint1.pt \
    --save-interval 5 \
    --save-dir "/content/drive/MyDrive/checkpoints/"
    --log-interval 1 \
    --log-format simple \
    --max-epoch 120 \
    --update-freq 16 \
    --max-update 30000 \
    --max-tokens 4000 \
    --arch transformer \
    --share-all-embeddings \
    --encoder-embed-dim 512 \
    --encoder-ffn-embed-dim 4096 \
    --encoder-attention-heads 8 \
    --encoder-layers 6 \
    --decoder-embed-dim 512 \
    --decoder-ffn-embed-dim 4096 \
    --decoder-attention-heads 8 \
    --decoder-layers 6 \
    --share-all-embeddings \
    --dropout 0.3 \
    --activation-fn gelu \
    --optimizer adam \
    --adam-betas '(0.9, 0.98)' \
    --lr 0.001 \
    --lr-scheduler inverse_sqrt \
    --scoring sacrebleu \
    --warmup-updates 4000 \
    --warmup-init-lr 1e-07 \
    --clip-norm 0.0 \
    --weight-decay 0.0 \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --task translation

2022-06-26 04:57:11 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.1, activation_fn='gelu', adam_betas='(0.9, 0.999)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.1, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=1.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='/Machine_Translation_Proto/data-bin', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=4096, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=8, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=True, decoder_output_dim=512, device_id=0, disable_valida

In [ ]:
# beam search
!fairseq-interactive data-bin \
    --buffer-size 1024 \
    --batch-size 128 \
    --path checkpoints/checkpoint2.pt \
    --lenpen 0.6 \
    --unkpen 0.6 \
    --beam 5 \
    < test.en \
    | grep '^H' \
    | cut -f 3 \
    | python src/decode.py \
    | tee output.txt \
    | sacrebleu /Machine_Translation_Proto/corpus/data/test.ja

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [ ]:
# top-p
!fairseq-interactive data-bin \
    --buffer-size 1024 \
    --batch-size 128 \
    --path checkpoints/checkpoint2.pt \
    --lenpen 0.6 \
    --unkpen 0.6 \
    --sampling \
    --beam 1 \
    --sampling-topp 0.8 \
    < test.en \
    | grep '^H' \
    | cut -f 3 \
    | python src/decode.py \
    | tee output.txt \
    | sacrebleu /Machine_Translation_Proto/corpus/data/test.ja

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 1.1 22.7/3.4/0.4/0.1 (BP = 1.000 ratio = 1.017 hyp_len = 117788 ref_len = 115852)


In [ ]:
!fairseq-interactive data-bin \
    --buffer-size 1024 \
    --batch-size 128 \
    --path checkpoints/checkpoint_last.pt \
    --lenpen 0.6 \
    --unkpen 0.6 \
    --beam 3 \
    < test.en \
    | grep '^H' \
    | cut -f 3 \
    | python src/decode.py \
    | tee output.txt \
    | sacrebleu /Machine_Translation_Proto/corpus/data/test.ja

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 36.3 65.1/42.9/29.8/22.1 (BP = 0.987 ratio = 0.987 hyp_len = 230721 ref_len = 233808)


In [ ]:
!head output.txt /Machine_Translation_Proto/corpus/data/test.ja

==> output.txt <==
 野球 の 好き な 生徒 も いれ ば 、 サッカー が 好き な 生徒 も いる 。
 トム は 私 たち の 援助 を 断っ た 。
 彼女 は 音楽 を 聴き ながら いつ も 勉強 する 。
 トム が いや だ と 言う の は 難しかっ た 。
 私 は 空 高く 飛ん で いる 鳥 の 群れ を 見 た 。
 彼 は 日本 の 社会 展開 に つい て 書い た 。
 その 人 たち が 最後 に 寄付 し た 本 は 、 実際 に は 、 人間 の 偉大 な 本 と 呼ば れる よう に なっ て いる 。
 勝利 は 決して 不 可能 で は ない 。
 多く の 外国 人 が 日本 に 働き に 来る 。
 彼 は そんな ばか な こと は し ない 。

==> /Machine_Translation_Proto/corpus/data/test.ja <==
野球 が 好き な 生徒 も いれ ば サッカー が 好き な 生徒 も いる 。 
トム は 僕 ら の 助け を 断っ た 。 
彼女 は 音楽 を 聴き ながら いつ も 勉強 する 。 
「 ノー 」 と 言う の は 、 トム に とっ て は 難しい こと だっ た 。 
鳥 の 群れ が 空 高く 飛ん で いる の を 見 まし た 。 
彼 は 日本 の 社会 的 発展 に つい て 書い た 。 
人間 の 心理 研究 に 対し て 不朽 の 貢献 を し た 書物 を 、 われわれ は 偉大 な 書物 を 呼ぶ の で ある 。 
勝利 は 得 られ そう も ない が 、 不 可能 で も ない 。 
多く の 外国 人 が 働く ため に 日本 に やっ て 来る 。 
彼 は そんな 馬鹿 な こと は し ない 。 


In [ ]:
import os
os.chdir("/Machine_Translation_Proto/src/")
from translation import Translation
en = input('英文を入力 > ')

tl = Translation("en", "ja")
tl.load(checkpoint_dir="/Machine_Translation_Proto/checkpoints/",
           checkpoint_file="checkpoint_last.pt",
           data_name_or_path="/Machine_Translation_Proto/data-bin/",
           path_bpe_model="/Machine_Translation_Proto/bpe.model",
          )
ja = tl.translate(en, 3, 0.6)
print(ja.replace('▁', ''))

英文を入力 > Yesterday I went to the park.
私は昨日公園に行きました。
